In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv()

llm = ChatOpenAI()

In [2]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in the following ways:\n\n1. Automated testing: Langsmith can provide tools and libraries for implementing automated testing, which can help speed up testing processes and ensure more thorough coverage of code.\n\n2. Test case generation: Langsmith can assist in generating test cases based on the specifications and requirements of the software being tested, making it easier to identify and address potential issues.\n\n3. Performance testing: Langsmith can provide tools and resources for performance testing, allowing testers to evaluate the performance of the software under different conditions and identify potential bottlenecks or areas for improvement.\n\n4. Integration testing: Langsmith can help with integration testing by providing tools and frameworks for testing how different components of the software interact and work together.\n\n5. Reporting and analysis: Langsmith can assist in generating test reports and analyzing test resu

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [4]:
chain = prompt | llm 

In [5]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content="Langsmith can help with testing in various ways:\n1. Automated Testing: Langsmith can automate the testing process to improve efficiency and accuracy. It can generate test scripts based on predefined test cases and execute them automatically.\n\n2. Test Data Generation: Langsmith can generate realistic and diverse test data to ensure comprehensive testing coverage. This helps in identifying edge cases and potential issues early in the development cycle.\n\n3. Performance Testing: Langsmith can simulate large loads and stress on the system to evaluate its performance under different conditions. It can generate realistic user interactions to test the system's responsiveness and scalability.\n\n4. Regression Testing: Langsmith can automate regression testing to quickly validate that new code changes have not introduced any unintended side effects or regressions in the software.\n\n5. Continuous Integration: Langsmith can be integrated into the CI/CD pipeline to automate

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [7]:
chain = prompt | llm | output_parser

In [8]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can help with testing in various ways, including:\n\n1. Automated Testing: Langsmith can be used to generate test cases, test data, and scripts for automated testing. This can help improve test coverage and efficiency.\n\n2. Test Scenario Generation: Langsmith can generate test scenarios based on specified constraints and requirements, helping testers to identify potential edge cases and corner cases that may not have been considered otherwise.\n\n3. API Testing: Langsmith can assist in generating API test scripts and data for testing the functionality and performance of APIs.\n\n4. Load Testing: Langsmith can be used to generate realistic load test scenarios to simulate high traffic conditions and test the scalability and performance of an application.\n\n5. Regression Testing: Langsmith can generate test cases to be used for regression testing, ensuring that new changes do not break existing functionality.\n\n6. Data-Driven Testing: Langsmith can generate test data sets fo

In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [11]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [13]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing users to visualize test results.'

In [14]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [15]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by providing capabilities to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It offers tracing and evaluation quick start guides, as well as additional resources such as the LangSmith Cookbook for tutorials and end-to-end walkthroughs.
